In [1]:
from bokeh.models import CategoricalColorMapper, NumeralTickFormatter, HoverTool
from bokeh.models import ColumnDataSource, Grid, LinearAxis, Plot, VBar
from bokeh.plotting import output_notebook, figure
from bokeh.io import reset_output, show, output_file
from bokeh.layouts import column, row
import numpy as np
import requests
import pandas as pd

In [2]:
larimer_vac_old_file = pd.read_csv('LC-COVID-vaccinations.csv')
larimer_vac_old_file

,Date,daily number of doses received by Larimer County residents,total number of doses recevied by residents,daily number of residents receiving first dose,total number of residents receiving first dose,daily number of residents vaccinated,total number of residents vaccinated,daily number of 70+ vaccinated,total number of 70+ vaccinated,daily number of 70+ at least one dose,total number of 70+ at least one dose,daily number of Latinx residents vaccinated,total of Latinx residents vaccinated,daily number of White non-Latinx residents vaccinated,total of White non-Latinx residents vaccinated,daily number of non-White non-Latinx residents vaccinated,total of non-White non-Latinx residents vaccinated,dailyUnknown,totalUnknown,mysteryColumn
0,12/14/2020,16,16,16,16,0,0,NaN,0,NaN,0,0.0,0,0.0,0,0.0,0,NaN,0,0.0
1,12/15/2020,4,20,3,19,1,1,0.0,0,1.0,1,NaN,0,1.0,1,NaN,0,NaN,0,0.0
2,12/16/2020,280,300,279,298,1,2,0.0,0,1.0,2,0.0,0,1.0,2,0.0,0,0.0,0,0.0
3,12/17/2020,939,1239,939,1237,0,2,0.0,0,10.0,12,0.0,0,0.0,2,0.0,0,0.0,0,0.0
4,12/18/2020,990,2229,989,2226,1,3,0.0,0,12.0,24,0.0,0,1.0,3,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,4/7/2021,4359,207056,1323,126509,3036,80547,72.0,29485,69.0,31371,116.0,2896,2593.0,72099,176.0,2550,151.0,3002,NaN
115,4/8/2021,6138,213194,2458,128967,3680,84227,140.0,29625,63.0,31434,142.0,3038,3197.0,75296,168.0,2718,173.0,3175,NaN
116,4/9/2021,5271,218465,1371,130338,3900,88127,82.0,29707,24.0,31458,172.0,3210,3366.0,78662,187.0,2905,175.0,3350,NaN
117,4/10/2021,4065,222530,1843,132181,2222,90349,69.0,29776,21.0,31479,103.0,3313,1880.0,80542,149.0,3054,90.0,3440,NaN


In [28]:
larimer_vac = pd.read_csv('https://speedtest.larimer.org/covid/index.php?file=vaccinations&csv')

larimer_cases = pd.read_csv('https://speedtest.larimer.org/covid/cases.csv')

larimer_deaths = pd.read_csv('https://larimer-county-data-lake.s3-us-west-2.amazonaws.com/Public/covid/covid_deaths.csv?t=1631890252549')

In [29]:
larimer_vac

,Date,daily number of doses received by Larimer County residents,total number of doses recevied by residents,daily number of residents receiving first dose,total number of residents receiving first dose,daily number of residents vaccinated,total number of residents vaccinated,daily number of 70+ vaccinated,total number of 70+ vaccinated,daily number of 70+ at least one dose,...,daily number of Latinx residents vaccinated,total of Latinx residents vaccinated,daily number of White non-Latinx residents vaccinated,total of White non-Latinx residents vaccinated,daily number of non-White non-Latinx residents vaccinated,total of non-White non-Latinx residents vaccinated,dailyUnknown,totalUnknown,daily_additional_doses,total_additional_doses
0,12/14/2020,31,31,23,23,8,8,1.0,1,0.0,...,0.0,0,8,8,0.0,0,0.0,0,0,0
1,12/15/2020,16,47,6,29,10,18,0.0,1,1.0,...,0.0,0,9,17,1.0,1,NaN,0,0,0
2,12/16/2020,309,356,298,327,11,29,0.0,1,1.0,...,1.0,1,9,26,1.0,2,0.0,0,0,0
3,12/17/2020,987,1343,970,1297,17,46,1.0,2,10.0,...,0.0,1,17,43,0.0,2,0.0,0,0,0
4,12/18/2020,1042,2385,1031,2328,11,57,2.0,4,14.0,...,0.0,1,11,54,0.0,2,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,9/11/2021,1584,425198,163,224772,190,210897,7.0,34218,23.0,...,16.0,11987,148,179143,16.0,11733,10.0,8034,1244,6541
272,9/12/2021,218,425416,98,224870,104,211001,0.0,34218,2.0,...,10.0,11997,68,179211,7.0,11740,19.0,8053,16,6557
273,9/13/2021,451,425867,202,225072,223,211224,9.0,34227,8.0,...,13.0,12010,176,179387,10.0,11750,24.0,8077,39,6596
274,9/14/2021,751,426618,154,225226,228,211452,12.0,34239,14.0,...,16.0,12026,175,179562,13.0,11763,24.0,8101,390,6986


In [5]:
display(larimer_vac.head())

display(larimer_cases.head())

display(larimer_deaths.head())

,Date,daily number of doses received by Larimer County residents,total number of doses recevied by residents,daily number of residents receiving first dose,total number of residents receiving first dose,daily number of residents vaccinated,total number of residents vaccinated,daily number of 70+ vaccinated,total number of 70+ vaccinated,daily number of 70+ at least one dose,...,daily number of Latinx residents vaccinated,total of Latinx residents vaccinated,daily number of White non-Latinx residents vaccinated,total of White non-Latinx residents vaccinated,daily number of non-White non-Latinx residents vaccinated,total of non-White non-Latinx residents vaccinated,dailyUnknown,totalUnknown,daily_additional_doses,total_additional_doses
0,12/14/2020,31,31,23,23,8,8,1.0,1,0.0,...,0.0,0,8,8,0.0,0,0.0,0,0,0
1,12/15/2020,16,47,6,29,10,18,0.0,1,1.0,...,0.0,0,9,17,1.0,1,NaN,0,0,0
2,12/16/2020,309,356,298,327,11,29,0.0,1,1.0,...,1.0,1,9,26,1.0,2,0.0,0,0,0
3,12/17/2020,987,1343,970,1297,17,46,1.0,2,10.0,...,0.0,1,17,43,0.0,2,0.0,0,0,0
4,12/18/2020,1042,2385,1031,2328,11,57,2.0,4,14.0,...,0.0,1,11,54,0.0,2,0.0,0,0,0


,CaseCount,ReportedDate,Sex,Age,Type,City
0,1,2020-03-09,Female,52.0,Confirmed,Johnstown
1,2,2020-03-15,Male,49.0,Confirmed,Fort Collins
2,3,2020-03-17,Female,53.0,Confirmed,Fort Collins
3,4,2020-03-17,Female,94.0,Confirmed,Loveland
4,5,2020-03-18,Male,49.0,Confirmed,Fort Collins


,death_id,death_date,age,gender,city,case_status,count
0,a0U5w00000edbfjEAA,2020-03-09,91,Female,Loveland,Probable,1
1,a0U5w00000edbfiEAA,2020-03-13,95,Female,Loveland,Probable,2
2,a0U5w00000edbfOEAQ,2020-03-15,90,Female,Loveland,Probable,3
3,a0U5w00000edbfJEAQ,2020-03-25,87,Female,Fort Collins,Confirmed,4
4,a0U5w00000edbfMEAQ,2020-03-25,74,Female,Loveland,Confirmed,5


In [6]:
daily_cases = larimer_cases.groupby(['ReportedDate']).size()
daily_deaths = larimer_deaths.groupby(['death_date']).size()


display(daily_cases)
display(daily_deaths)

ReportedDate
2020-03-09      1
2020-03-15      1
2020-03-17      2
2020-03-18      1
2020-03-19      2
             ... 
2021-09-13    110
2021-09-14     86
2021-09-15    127
2021-09-16    150
2021-09-17     91
Length: 551, dtype: int64

death_date
2020-03-09    1
2020-03-13    1
2020-03-15    1
2020-03-25    2
2020-03-29    2
             ..
2021-09-08    2
2021-09-09    1
2021-09-11    2
2021-09-14    2
2021-09-15    1
Length: 178, dtype: int64

In [7]:
larimer_cases.ReportedDate.dtype

dtype('O')

In [8]:
larimer_vac.shape

(276, 21)

In [9]:
len(larimer_vac)

276

In [10]:
larimer_vac.count()

Date                                                          276
daily number of doses received by Larimer County residents    276
total number of doses recevied by residents                   276
daily number of residents receiving first dose                276
total number of residents receiving first dose                276
daily number of residents vaccinated                          276
total number of residents vaccinated                          276
daily number of 70+ vaccinated                                274
total number of 70+ vaccinated                                276
daily number of 70+ at least one dose                         274
total number of 70+ at least one dose                         276
daily number of Latinx residents vaccinated                   275
total of Latinx residents vaccinated                          276
daily number of White non-Latinx residents vaccinated         276
total of White non-Latinx residents vaccinated                276
daily numb

In [11]:
larimer_vac.describe()

,daily number of doses received by Larimer County residents,total number of doses recevied by residents,daily number of residents receiving first dose,total number of residents receiving first dose,daily number of residents vaccinated,total number of residents vaccinated,daily number of 70+ vaccinated,total number of 70+ vaccinated,daily number of 70+ at least one dose,total number of 70+ at least one dose,daily number of Latinx residents vaccinated,total of Latinx residents vaccinated,daily number of White non-Latinx residents vaccinated,total of White non-Latinx residents vaccinated,daily number of non-White non-Latinx residents vaccinated,total of non-White non-Latinx residents vaccinated,dailyUnknown,totalUnknown,daily_additional_doses,total_additional_doses
count,276.000000,276.000000,276.00000,276.000000,276.000000,276.000000,274.000000,276.000000,274.000000,276.000000,275.000000,276.000000,276.000000,276.000000,274.000000,276.000000,271.000000,276.000000,276.000000,276.000000
mean,1547.695652,246935.536232,816.32971,138746.797101,766.532609,116649.416667,124.992701,24381.626812,129.967153,28186.192029,43.770909,5960.576087,650.876812,100891.268116,42.948905,5981.275362,29.948339,3816.297101,26.619565,353.329710
std,1433.618167,155267.969209,869.17116,80584.120169,854.846391,81843.170180,264.682614,13204.865825,257.007723,11294.835958,48.306071,4601.817644,739.721967,69669.187646,53.071839,4674.276506,30.286733,3018.466093,122.987199,1265.620283
min,2.000000,31.000000,2.00000,23.000000,0.000000,8.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,451.000000,84787.750000,221.00000,57435.250000,175.250000,27245.750000,6.000000,13716.000000,8.000000,27901.250000,13.000000,1032.750000,132.500000,24587.500000,11.250000,952.750000,12.000000,672.750000,0.000000,0.000000
50%,935.000000,307654.000000,411.50000,179957.500000,358.500000,139657.500000,14.000000,32188.000000,16.000000,34088.000000,28.000000,6386.500000,285.000000,122992.000000,22.000000,6479.000000,20.000000,3800.000000,0.000000,0.000000
75%,2520.500000,391766.500000,1098.00000,209423.250000,1075.250000,197769.500000,87.750000,33735.500000,94.500000,34919.750000,54.000000,10722.750000,945.750000,169310.750000,49.000000,10862.000000,37.000000,6874.000000,0.000000,0.000000
max,6677.000000,427164.000000,4492.00000,225307.000000,4162.000000,211563.000000,1750.000000,34248.000000,1595.000000,35611.000000,311.000000,12037.000000,3616.000000,179642.000000,243.000000,11768.000000,159.000000,8116.000000,1244.000000,7347.000000


In [12]:
print(larimer_vac.Date.dtype)


object


In [13]:
real_date = pd.to_datetime(larimer_vac.Date)
real_date

0     2020-12-14
1     2020-12-15
2     2020-12-16
3     2020-12-17
4     2020-12-18
         ...    
271   2021-09-11
272   2021-09-12
273   2021-09-13
274   2021-09-14
275   2021-09-15
Name: Date, Length: 276, dtype: datetime64[ns]

In [14]:
larimer_vac['Date'] = real_date
larimer_vac

,Date,daily number of doses received by Larimer County residents,total number of doses recevied by residents,daily number of residents receiving first dose,total number of residents receiving first dose,daily number of residents vaccinated,total number of residents vaccinated,daily number of 70+ vaccinated,total number of 70+ vaccinated,daily number of 70+ at least one dose,...,daily number of Latinx residents vaccinated,total of Latinx residents vaccinated,daily number of White non-Latinx residents vaccinated,total of White non-Latinx residents vaccinated,daily number of non-White non-Latinx residents vaccinated,total of non-White non-Latinx residents vaccinated,dailyUnknown,totalUnknown,daily_additional_doses,total_additional_doses
0,2020-12-14,31,31,23,23,8,8,1.0,1,0.0,...,0.0,0,8,8,0.0,0,0.0,0,0,0
1,2020-12-15,16,47,6,29,10,18,0.0,1,1.0,...,0.0,0,9,17,1.0,1,NaN,0,0,0
2,2020-12-16,309,356,298,327,11,29,0.0,1,1.0,...,1.0,1,9,26,1.0,2,0.0,0,0,0
3,2020-12-17,987,1343,970,1297,17,46,1.0,2,10.0,...,0.0,1,17,43,0.0,2,0.0,0,0,0
4,2020-12-18,1042,2385,1031,2328,11,57,2.0,4,14.0,...,0.0,1,11,54,0.0,2,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,2021-09-11,1584,425198,163,224772,190,210897,7.0,34218,23.0,...,16.0,11987,148,179143,16.0,11733,10.0,8034,1244,6541
272,2021-09-12,218,425416,98,224870,104,211001,0.0,34218,2.0,...,10.0,11997,68,179211,7.0,11740,19.0,8053,16,6557
273,2021-09-13,451,425867,202,225072,223,211224,9.0,34227,8.0,...,13.0,12010,176,179387,10.0,11750,24.0,8077,39,6596
274,2021-09-14,751,426618,154,225226,228,211452,12.0,34239,14.0,...,16.0,12026,175,179562,13.0,11763,24.0,8101,390,6986


In [15]:
larimer_vac.Date.dtype

dtype('<M8[ns]')

In [17]:
for col in larimer_vac.columns:
    print(col)

Date
daily number of doses received by Larimer County residents
total number of doses recevied by residents
daily number of residents receiving first dose
total number of residents receiving first dose
daily number of residents vaccinated
total number of residents vaccinated
daily number of 70+ vaccinated
total number of 70+ vaccinated
daily number of 70+ at least one dose
total number of 70+ at least one dose
daily number of Latinx residents vaccinated
total of Latinx residents vaccinated
daily number of White non-Latinx residents vaccinated
total of White non-Latinx residents vaccinated
daily number of non-White non-Latinx residents vaccinated
total of non-White non-Latinx residents vaccinated
dailyUnknown
totalUnknown
daily_additional_doses
total_additional_doses


In [18]:
larimer_vac['daily number of doses received by Larimer County residents'].dtype

dtype('int64')

In [19]:
# make a backup file from the source
larimer_vac.to_csv('larimer_vac_backup.csv')

In [20]:
larimer_vac[['Date','daily number of doses received by Larimer County residents']]

,Date,daily number of doses received by Larimer County residents
0,2020-12-14,31
1,2020-12-15,16
2,2020-12-16,309
3,2020-12-17,987
4,2020-12-18,1042
...,...,...
271,2021-09-11,1584
272,2021-09-12,218
273,2021-09-13,451
274,2021-09-14,751


## Quantify missing data

In [21]:
valid_entries = larimer_vac.count()
total_rows = len(larimer_vac.index)
missing_data = total_rows - valid_entries
missing_data.head()

Date                                                          0
daily number of doses received by Larimer County residents    0
total number of doses recevied by residents                   0
daily number of residents receiving first dose                0
total number of residents receiving first dose                0
dtype: int64

In [23]:
#lar_vac_data = ColumnDataSource(larimer_vac)

reset_output()
output_notebook()

x = larimer_vac['Date']
top = larimer_vac['daily number of doses received by Larimer County residents']

daily_vac_figure = figure(title="Daily Vaccinations",
                         x_axis_type="datetime")

daily_vac_figure.vbar(x=x,
               top=top,
               width=0.9)



show(daily_vac_figure)

Loading BokehJS ...

In [26]:
lar_vac_data = ColumnDataSource(larimer_vac)

reset_output()
output_notebook()

# x = lar_vac_data['Date']
# y = lar_vac_data('daily number of doses received by Larimer County residents')

daily_vac_figure = figure(title='Daily Vaccinations',
                         x_axis_type="datetime")

daily_vac_figure.line(x='Date',
                   y='daily number of doses received by Larimer County residents',
               source=lar_vac_data)



show(daily_vac_figure)

Loading BokehJS ...